<a href="https://colab.research.google.com/github/Krishanx2003/MachineLearning/blob/main/Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'd5573aa2c8c14defa7ff4c630f3ae394'
CLIENT_SECRET = 'd2cc24e571e24152ba15d4d86cb6e39b'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 8.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                                           Track Name  \
0                                              redrum   
1                                         Lovin On Me   
2                                     I'm Good (Blue)   
3                                               Prada   
4                                            BELLAKEO   
..                                                ...   
95                               Drugs From Amsterdam   
96                        Another Love - Tiësto Remix   
97  WORTH NOTHING (feat. Oliver Tree) - Fast & Fur...   
98                                           Remember   
99                                  MONEY ON THE DASH   

                                              Artists  \
0                                           21 Savage   
1                                         Jack Harlow   
2                            David Guetta, Bebe Rexha   
3                         cassö, RAYE, D-Block Europe   
4                             

In [ ]:
playlist_id = '37i9dQZF1E39oYPd0mdipS'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1E39oYPd0mdipS/tracks with Params: {'limit': 100, 'offset': 0, 'fields': 'items(track(id, name, artists, album(id, name)))', 'market': None, 'additional_types': 'track'} returned 404 due to Not found.


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/playlists/37i9dQZF1E39oYPd0mdipS/tracks?limit=100&offset=0&fields=items%28track%28id%2C+name%2C+artists%2C+album%28id%2C+name%29%29%29&additional_types=track:
 Not found., reason: None

In [ ]:
print(music_df.isnull().sum())

In [ ]:
playlist_id = '37i9dQZF1DXbYM3nMM0oPk'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [ ]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

def calculate_weighted_popularity(release_date):
    """
    Function to calculate weighted popularity scores based on release date
    """
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    time_span = datetime.now() - release_date
    weight = 1 / (time_span.days + 1)
    return weight

def content_based_recommendations(input_song_name, music_df, music_features_scaled, num_recommendations=5):
    """
    Function to get content-based recommendations based on music features
    """
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return None

    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]
    return content_based_recommendations

def hybrid_recommendations(input_song_name, music_df, music_features_scaled, alpha=0.5, num_recommendations=5):
    """
    Function to get hybrid recommendations based on weighted popularity
    """
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return None

    content_based_rec = content_based_recommendations(input_song_name, music_df, music_features_scaled, num_recommendations)
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    hybrid_recommendations = pd.concat([content_based_rec, pd.DataFrame([{
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }])])

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]
    return hybrid_recommendations


input_song_name = "Memories"
recommendations = hybrid_recommendations(input_song_name, music_df, music_features_scaled, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)



'Memories' not found in the dataset. Please enter a valid song name.
Hybrid recommended songs for 'Memories':
None


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id='d5573aa2c8c14defa7ff4c630f3ae394', client_secret='d2cc24e571e24152ba15d4d86cb6e39b')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def recommend_playlists(seed_playlist_id, num_recommendations=5):
    """
    Function to recommend playlists based on a seed playlist.
    """
    try:
        # Get tracks from the seed playlist
        seed_playlist_tracks = sp.playlist_tracks(seed_playlist_id, fields='items(track(id))')
        track_ids = [track['track']['id'] for track in seed_playlist_tracks['items'] if track.get('track')]

        if not track_ids:
            print("No tracks found in the seed playlist.")
            return []

        # Get audio features for each track in the seed playlist
        audio_features = sp.audio_features(track_ids)

        # Filter out non-numeric features and calculate the mean of each audio feature
        numeric_features = {feature: [track[feature] for track in audio_features if isinstance(track.get(feature), (int, float))] for feature in audio_features[0].keys() if feature != 'id'}
        mean_features = {feature: sum(values) / len(values) for feature, values in numeric_features.items()}

        # Search for playlists similar to the seed playlist based on mean audio features
        results = sp.recommendations(seed_genres=[], seed_tracks=track_ids, limit=num_recommendations, **mean_features)

        recommended_playlists = []
        for playlist in results['tracks']:
            recommended_playlists.append({
                'Playlist Name': playlist['name'],
                'Playlist ID': playlist['id'],
                'Owner': playlist['owner']['display_name']
            })

        return recommended_playlists

    except spotipy.SpotifyException as e:
        print(f"Error occurred: {e}")
        return []

# Example usage
seed_playlist_id = '37i9dQZF1DXbYM3nMM0oPk'
recommendations = recommend_playlists(seed_playlist_id, num_recommendations=5)
print("Recommended Playlists:")
for playlist in recommendations:
    print(f"Name: {playlist['Playlist Name']}, ID: {playlist['Playlist ID']}, Owner: {playlist['Owner']}")


ZeroDivisionError: division by zero